In [ ]:
# import cv2
# import mediapipe as mp

In [ ]:
# mp_hand = mp.solutions.hands#Landmarks
# mp_draw = mp.solutions.drawing_utils#drawing landmarks

In [ ]:
# data = mp_hand.Hands(1)

In [1]:
import cv2

In [20]:
import cv2
import mediapipe as mp
import pyautogui
import mss
import mss.tools
import ctypes
import tkinter as tk
import keyboard
import time
import sys

# --- STEP 1: DPI ACCURACY ---
try:
    ctypes.windll.shcore.SetProcessDpiAwareness(1)
except Exception:
    try:
        ctypes.windll.user32.SetProcessDPIAware()
    except:
        pass

# --- STEP 2: OVERLAY UI ---
class Overlay:
    def __init__(self):
        self.root = tk.Tk()
        self.root.overrideredirect(True)
        self.root.attributes("-topmost", True)
        self.root.attributes("-transparentcolor", "white")
        self.root.config(bg='white')
        self.root.wm_attributes("-disabled", True)
        
        self.canvas = tk.Canvas(self.root, width=self.root.winfo_screenwidth(), 
                               height=self.root.winfo_screenheight(), 
                               bg='white', highlightthickness=0)
        self.canvas.pack()
        
        self.rect = self.canvas.create_rectangle(0, 0, 0, 0, outline="green", width=3)
        self.label = self.canvas.create_text(0, 0, text="", fill="green", font=("Arial", 14, "bold"), anchor="sw")
        
    def update_box(self, x1, y1, x2, y2, state="moving", countdown="", flash_now=False):
        try:
            self.canvas.coords(self.rect, x1, y1, x2, y2)
            self.canvas.coords(self.label, x1, y1 - 5)
            self.canvas.itemconfig(self.label, text=countdown)

            if flash_now:
                self.canvas.itemconfig(self.rect, fill="white")
            else:
                self.canvas.itemconfig(self.rect, fill="")
            
            if state == "locked":
                self.canvas.itemconfig(self.rect, outline="red")
                self.canvas.itemconfig(self.label, fill="red")
            elif state == "closing":
                self.canvas.itemconfig(self.rect, outline="orange")
                self.canvas.itemconfig(self.label, fill="orange")
            else:
                self.canvas.itemconfig(self.rect, outline="green")
                self.canvas.itemconfig(self.label, fill="green")

            self.root.update()
        except:
            pass

    def close(self):
        try:
            self.root.destroy()
        except:
            pass

overlay = Overlay()

# --- STEP 3: COORDINATES & FLAGS ---
sct = mss.mss()
try:
    main_monitor = sct.monitors[1]
except IndexError:
    main_monitor = sct.monitors[0]

current_box = [0, 0, 0, 0] 
capture_requested = False  
is_running = True 
is_locked = False

last_move_time = time.time()
fist_start_time = None
lock_threshold = 20  
lock_delay = 3.0     

def reset_lock():
    global is_locked, last_move_time
    is_locked = False
    last_move_time = time.time()
    print("Lock Released. Resuming tracking...")

def request_capture():
    global capture_requested
    capture_requested = True

def stop_program():
    global is_running
    is_running = False

# Hotkeys
keyboard.add_hotkey('r', reset_lock) # PRESS 'R' TO MOVE BOX AGAIN
keyboard.add_hotkey('s', request_capture)
keyboard.add_hotkey('q', stop_program)

# --- STEP 4: VISION BRAIN ---
screen_w, screen_h = pyautogui.size()
video = cv2.VideoCapture(0)
mp_hand = mp.solutions.hands
hands_detector = mp_hand.Hands(max_num_hands=1, min_detection_confidence=0.8)

p_t_x, p_t_y, p_i_x, p_i_y = 0, 0, 0, 0
smooth_factor = 0.15

print("AR Sniper: PERMANENT LOCK MODE")
print("- Hold still 3s to FREEZE FOREVER")
print("- Press 'R' to UNLOCK and move again")
print("- Press 'Q' to EXIT")

try:
    while is_running:
        success, img = video.read()
        if not success: break
        
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands_detector.process(img_rgb)

        display_text = ""
        current_state = "moving"
        flash_effect = False

        # --- LOGIC BRANCH: LOCKED VS TRACKING ---
        if is_locked:
            # COMPLETELY IGNORE HANDS FOR MOVEMENT
            current_state = "locked"
            display_text = "LOCKED - Press 'R' to reset"
        else:
            # ONLY RUN TRACKING IF NOT LOCKED
            if result.multi_hand_landmarks:
                hand_lms = result.multi_hand_landmarks[0]
                
                # Fist detection for Exit
                is_fist = (hand_lms.landmark[8].y > hand_lms.landmark[6].y and
                           hand_lms.landmark[12].y > hand_lms.landmark[10].y and
                           hand_lms.landmark[16].y > hand_lms.landmark[14].y and
                           hand_lms.landmark[20].y > hand_lms.landmark[18].y)

                if is_fist:
                    fist_start_time = fist_start_time or time.time()
                    if time.time() - fist_start_time > 2.0: is_running = False
                    display_text, current_state = "EXITING...", "closing"
                else:
                    fist_start_time = None 
                    thumb = hand_lms.landmark[4]
                    index = hand_lms.landmark[8]
                    target_tx, target_ty = thumb.x * screen_w, thumb.y * screen_h
                    target_ix, target_iy = index.x * screen_w, index.y * screen_h

                    movement = abs(target_tx - p_t_x) + abs(target_ty - p_t_y)

                    if movement > lock_threshold:
                        last_move_time = time.time()
                    else:
                        time_elapsed = time.time() - last_move_time
                        if time_elapsed > lock_delay:
                            is_locked = True
                            flash_effect = True
                            request_capture() 
                        else:
                            display_text = f"Locking in {max(0, int(lock_delay - time_elapsed) + 1)}..."

                    # Update smoothing and current_box ONLY while tracking
                    p_t_x += (target_tx - p_t_x) * smooth_factor
                    p_t_y += (target_ty - p_t_y) * smooth_factor
                    p_i_x += (target_ix - p_i_x) * smooth_factor
                    p_i_y += (target_iy - p_i_y) * smooth_factor
                    
                    current_box = [int(min(p_t_x, p_i_x)), int(min(p_t_y, p_i_y)), 
                                   int(max(p_t_x, p_i_x)), int(max(p_t_y, p_i_y))]

        # --- STEP 5: CAPTURE & OVERLAY ---
        if capture_requested:
            overlay.update_box(*current_box, state="locked", flash_now=True)
            x1, y1, x2, y2 = current_box
            width, height = x2 - x1, y2 - y1
            if width > 20 and height > 20:
                region = {"top": max(0, main_monitor["top"] + y1), "left": max(0, main_monitor["left"] + x1), "width": width, "height": height}
                sct_img = sct.grab(region)
                mss.tools.to_png(sct_img.rgb, sct_img.size, output=f"cap_{int(time.time())}.png")
            capture_requested = False 
        else:
            overlay.update_box(*current_box, state=current_state, countdown=display_text, flash_now=flash_effect)

        time.sleep(0.01)

except Exception as e:
    print(f"Error: {e}")
finally:
    video.release()
    overlay.close()
    keyboard.unhook_all()
    sys.exit()


AR Sniper: PERMANENT LOCK MODE
- Hold still 3s to FREEZE FOREVER
- Press 'R' to UNLOCK and move again
- Press 'Q' to EXIT
Lock Released. Resuming tracking...


SystemExit: 